In [ ]:
# Capture video
# Classify emotion
# Output the emotion

import cv2
import dlib
import numpy as np
import os
import math
from sklearn.externals import joblib

# Function that returns the distance between two given points
def ptDist(pt1, pt2):
    x1 = pt1[0]
    x2 = pt2[0]
    y1 = pt1[1]
    y2 = pt2[1]
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Function that eturns normalized array of Euclidean distances given two arrays of coordinates
def getEucDist(nLA, eLA):
    LDA = []
    sum = 0
    # Get array of distances
    for pt in range(68):
        dif = ptDist(eLA[pt], nLA[pt])     
        LDA.append(dif)
        sum += dif
    avg = sum/68
    # Normalize array
    for i in range(68):
        LDA[i] -= avg      
    return LDA

# Method to get 68 landmarks for IMAGE passed to the function
def get17LMA(img):
    predictor_path = "/Users/kaili/dlib/examples/build/sp.dat"
    detector = dlib.fhog_object_detector("/Users/kaili/dlib/examples/build/face_detector.svm")
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    
    # Detect face(s)
    dets = detector(img, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    if len(dets)>0:
        shape = predictor(img, dets[0])
        for index in range(0, 17):
            point = shape.part(index)
            tuplepoint = (point.x, point.y)
            LMA.append(tuplepoint)
        return LMA

# Function that returns 13-point numpy vector of distances between selected physiological pts, given the 68-point vector
def getPhysDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[2], iLA[3])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[2], iLA[4])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[3], iLA[4])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[2], iLA[6])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[3], iLA[7])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[5], iLA[11])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[8], iLA[12])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[11], iLA[12])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[15], iLA[16])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[5], iLA[13])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[7], iLA[14])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[0], iLA[9])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[1], iLA[10])) #distance 12: brow arch- right
    return np.asarray(PDA)

    
cap = cv2.VideoCapture(0) # capture video from computer camera
cap.set(3,640)
cap.set(4,480)
loaded = joblib.load('theModel.pkl')
detector = dlib.fhog_object_detector("/Users/kaili/dlib/examples/build/face_detector.svm")
predictor_path = "/Users/kaili/dlib/examples/build/sp.dat"
predictor = dlib.shape_predictor(predictor_path)
distances = []
baseline = cv2.imread("baseline.jpg", 1)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    dets = detector(frame, 1)
    x_list = list()
    y_list = list()
    
    for d in dets:
        shape = predictor(frame, d)
        for index in range(0, 17):
            point = shape.part(index)
            x_list.append(point.x)
            y_list.append(point.y)
    
    for i in range((len(dets)*17)):
        cv2.circle(frame, (x_list[i], y_list[i]), 3, (0, 255, 0), -1)
    
    for rect in dets:
        cv2.rectangle(frame, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
    cv2.imshow('frame',frame)

    # Get physiological feature vector for current frame
    distances.append(getPhysDA(get17LMA(baseline)))
    if get17LMA(frame) is not None:
        distances.append(getPhysDA(get17LMA(frame)))
        print(getPhysDA(get17LMA(frame))) #print distances
    
    # Classify expression in image
    if len(distances)>1:
        prediction = loaded.predict_proba(np.asarray(distances[0]-distances[-1])) # Get array of probabilities
        print(prediction)
        index = np.argmax(prediction)
        
    # Print the emotion corresponding to the position
        if index==0:
            print("Anger")
            #os.system("ssh pi@192.168.1.108 python emotions/anger.py")
        elif index==2:
            print("Disgust")
        elif index==3:
            print("Fear")
        elif index==4:
            print("Happiness")
            #os.system("ssh pi@192.168.1.108 python emotions/happiness.py")
        elif index==5:
            print("Sadness")
            #os.system("ssh pi@192.168.1.108 python emotions/sadness.py")
        elif index==6:
            print("Surprise")
            #os.system("ssh pi@192.168.1.108 python emotions/surprise.py")
            
    # Break out of while loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break      
        
# Release video capture
cap.release()
cv2.destroyAllWindows()

[  40.01249805   34.66987165   35.           35.17101079   42.19004622
   83.72574276  102.23991393  181.40837908   12.04159458  115.76700739
  100.12492197   24.0208243    24.0208243 ]
[[ 0.2528293   0.08447764  0.25338739  0.04220198  0.25318089  0.0506346
   0.0632882 ]]


/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[[ 0.24611247  0.1893309   0.13639572  0.07313549  0.03193525  0.15520271
   0.16788745]]


/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[[ 0.24611247  0.1893309   0.13639572  0.07313549  0.03193525  0.15520271
   0.16788745]]
[  61.03277808   38.20994635   43.60045871   36.68787266   46.04345773
  140.43147795  165.00303028  245.4893073    17.          160.33090781
  141.12760184   34.0147027    42.19004622]
[[ 0.2528293   0.08447764  0.25338739  0.04220198  0.25318089  0.0506346
   0.0632882 ]]


...

/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[  60.03332408   39.05124838   44.82186966   38.89730068   48.0416486
  139.58868149  163.          247.63077353   17.          161.55494421
  142.05632686   35.0142828    41.10960958]
[[ 0.2528293   0.08447764  0.25338739  0.04220198  0.25318089  0.0506346
   0.0632882 ]]


/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[  61.03277808   38.94868419   44.20407221   38.60051813   47.04253395
  141.42135624  164.01219467  242.78797334   18.          160.33090781
  142.0880009    35.0142828    44.18144407]
[[ 0.2528293   0.08447764  0.25338739  0.04220198  0.25318089  0.0506346
   0.0632882 ]]


/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [ ]:
print(index)

In [2]:
##########################




# TRAIN THE REAL DEAL PHYSIOLOGICAL MODEL






###########################



# Get array of Euclidean differences between neutral and expression 68-landmark arrays
for imgIndex in range(len(dataset)):
    NLA.append(get68LMA(dataset[imgIndex][0]))
    ELA.append(get68LMA(dataset[imgIndex][1]))

"""PHYSIOLOGICALLY BASED LANDMARK DETECTION MODEL"""
# Get array of corresponding differences between 13 distances of base images and expression images
tempPDA = []
for i in range(len(NLA)):
    tempPDA.append(getPDA(NLA[i])-getPDA(ELA[i]))
PDA = np.asmatrix(tempPDA)

# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train, x_test, y_train, y_test = cross_validation.train_test_split(PDA, targetValues, test_size=0.4, random_state=3)
CLF = svm.SVC(kernel='poly',probability=True).fit(x_train, y_train)
predictions = CLF.predict(x_test)
cMatrix = confusion_matrix(y_test, predictions)

# Format CMatrix2 to show percentages, already rounded to 1 decimal point
# rowSums2 = np.sum(cMatrix2, axis=1)
# pCMatrix2 = np.divide(cMatrix2*100., rowSums2, dtype=float)
print(cMatrix)

joblib.dump(CLF, 'theModel.pkl') 

[[11  2  1  0  0  0  1]
 [ 0  7  0  0  0  0  0]
 [ 1  0 19  0  0  0  0]
 [ 1  1  0  7  1  1  0]
 [ 0  0  1  0 27  0  0]
 [ 1  5  0  2  1  6  0]
 [ 0  2  0  2  0  0 31]]


['theModel.pkl',
 'theModel.pkl_01.npy',
 'theModel.pkl_02.npy',
 'theModel.pkl_03.npy',
 'theModel.pkl_04.npy',
 'theModel.pkl_05.npy',
 'theModel.pkl_06.npy',
 'theModel.pkl_07.npy',
 'theModel.pkl_08.npy',
 'theModel.pkl_09.npy',
 'theModel.pkl_10.npy',
 'theModel.pkl_11.npy']